In [ ]:
# Copyright (c) Meta Platforms, Inc. and affiliates.

In [1]:
import sys

# import sam-3d-objects code
sys.path.append("/data/machine_learning/cpx/sam-3d-objects")

In [2]:
import os
import shutil

# ============================================
# 重要：必须在导入任何使用 torch cpp_extension 的模块之前设置环境变量
# ============================================

# 正确设置环境变量 PATH（需要先获取当前 PATH，然后拼接）
current_path = os.environ.get("PATH", "")
cuda_bin = "/data/CUDA/cuda-12.4/bin"
venv_bin = "/data/machine_learning/cpx/sam-3d-objects/.venv/bin"

# 将 CUDA bin 和 venv bin 添加到 PATH（确保 ninja 可执行文件能被找到）
# 注意：将 venv_bin 放在最前面，确保优先使用虚拟环境中的工具
new_path = f"{venv_bin}:{cuda_bin}:{current_path}"
os.environ["PATH"] = new_path

# 设置 CUDA 相关环境变量
os.environ["TORCH_CUDA_ARCH_LIST"] = "8.0"
os.environ["CUDA_HOME"] = "/data/CUDA/cuda-12.4"
os.environ["MAX_JOBS"] = "8"

# 验证关键工具是否在 PATH 中
print("=" * 60)
print("环境变量设置验证")
print("=" * 60)
ninja_path = shutil.which("ninja")
nvcc_path = shutil.which("nvcc")
print(f"✓ ninja found at: {ninja_path}")
print(f"✓ nvcc found at: {nvcc_path}")
if not ninja_path:
    print("⚠ WARNING: ninja not found in PATH! This may cause gsplat compilation to fail.")
if not nvcc_path:
    print("⚠ WARNING: nvcc not found in PATH! This may cause CUDA compilation to fail.")

# 验证 torch 是否能检测到 ninja
# 注意：如果 torch 已经在之前的 cell 中被导入，可能需要重启内核
try:
    import torch.utils.cpp_extension
    # 强制重新检查 ninja 可用性
    is_available = torch.utils.cpp_extension.is_ninja_available()
    print(f"✓ torch.utils.cpp_extension.is_ninja_available(): {is_available}")
    if not is_available:
        print("⚠ WARNING: torch cannot detect ninja!")
        print("⚠ 如果 torch 在设置 PATH 之前已被导入，请重启 Jupyter 内核后重新运行。")
    else:
        # 尝试调用 verify_ninja_availability 来确保它能正常工作
        try:
            torch.utils.cpp_extension.verify_ninja_availability()
            print("✓ torch.utils.cpp_extension.verify_ninja_availability() passed")
        except RuntimeError as e:
            print(f"✗ torch.utils.cpp_extension.verify_ninja_availability() failed: {e}")
            print("⚠ 请重启 Jupyter 内核后重新运行。")
except ImportError:
    print("ℹ torch 尚未导入，这是正常的。环境变量将在导入时生效。")
except Exception as e:
    print(f"⚠ WARNING: Could not verify ninja availability: {e}")

print("=" * 60)


环境变量设置验证
✓ ninja found at: /data/machine_learning/cpx/sam-3d-objects/.venv/bin/ninja
✓ nvcc found at: /data/CUDA/cuda-12.4/bin/nvcc
✓ torch.utils.cpp_extension.is_ninja_available(): True
✓ torch.utils.cpp_extension.verify_ninja_availability() passed


## 1. Imports and Model Loading

In [3]:
import os
import uuid
import imageio
import numpy as np
from IPython.display import Image as ImageDisplay

from inference import Inference, ready_gaussian_for_video_rendering, load_image, load_masks, display_image, make_scene, render_video, interactive_visualizer

2026-01-16 11:56:44.524 | INFO     | sam3d_objects.pipeline.inference_pipeline:set_attention_backend:17 - GPU name is NVIDIA A800 80GB PCIe


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


2026-01-16 11:56:45.781 | INFO     | sam3d_objects.model.backbone.tdfy_dit.modules.sparse:__from_env:39 - [SPARSE] Backend: spconv, Attention: flash_attn
2026-01-16 11:56:49.801 | INFO     | sam3d_objects.model.backbone.tdfy_dit.modules.attention:__from_env:30 - [ATTENTION] Using backend: flash_attn


[SPARSE][CONV] spconv algo: auto


2026-01-16 11:56:50.564 | WARNING  | sam3d_objects.data.dataset.tdfy.preprocessor:__post_init__:51 - No rgb pointmap normalizer provided, using scale + shift 
2026-01-16 11:56:50.565 | WARNING  | sam3d_objects.data.dataset.tdfy.preprocessor:__post_init__:51 - No rgb pointmap normalizer provided, using scale + shift 


In [ ]:
config_path = "/data/models/LLM-models-file/sam-3d-objects/checkpoints/pipeline.yaml"
inference = Inference(config_path, compile=False)

## 2. Load input image to lift to 3D (multiple objects)

加载输入图像和 Mask，以便提升为 3D（多个对象）

In [ ]:
IMAGE_PATH = "./images/shutterstock_stylish_kidsroom_1640806567/image.png"
IMAGE_NAME = os.path.basename(os.path.dirname(IMAGE_PATH))

image = load_image(IMAGE_PATH)
masks = load_masks(os.path.dirname(IMAGE_PATH), extension=".png")
display_image(image, masks)

## 3. Generate Gaussian Splats

生成高斯溅射（一种 3D 渲染方法）

In [ ]:
outputs = [inference(image, mask, seed=42) for mask in masks]

# for index, output in enumerate(outputs):
#     # 导出高斯溅射
#     output["gs"].save_ply(f"./gaussians/single/{IMAGE_NAME}_{index}.ply")

In [ ]:
print(f"length of outputs: {len(outputs)}")

## 4. Visualize Gaussian Splat of the Scene
### a. Animated Gif

In [ ]:
scene_gs = make_scene(*outputs)
# export posed gaussian splatting (as point cloud)
scene_gs.save_ply(f"./gaussians/{IMAGE_NAME}_posed.ply")

scene_gs = ready_gaussian_for_video_rendering(scene_gs)
# export gaussian splatting (as point cloud)
scene_gs.save_ply(f"./gaussians/multi/{IMAGE_NAME}.ply")

video = render_video(
    scene_gs,
    r=1,
    fov=60,
    resolution=512,
)["color"]

# save video as gif
imageio.mimsave(
    os.path.join(f"./gaussians/multi/{IMAGE_NAME}.gif"),
    video,
    format="GIF",
    duration=1000 / 30,  # default assuming 30fps from the input MP4
    loop=0,  # 0 means loop indefinitely
)

# notebook display
ImageDisplay(url=f"gaussians/multi/{IMAGE_NAME}.gif?cache_invalidator={uuid.uuid4()}",)

### b. Interactive Visualizer

In [ ]:
# might take a while to load (black screen)
interactive_visualizer(f"./gaussians/multi/{IMAGE_NAME}.ply")